<a href="https://colab.research.google.com/github/tirals88/Numerical-Mathematics-and-Computing/blob/main/Chap8_More%20on%20Linear%20Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

%matplotlib inline

# 8.1 테일러 급수법

편미분방정식을 포함한 응용분야에서는 Sparse matrix를 가지는 큰 선형 시스템이 등장한다.

가우스 소거법은 0인 성분들을 0이 아닌 값들로 채울 수 있는 반면, 반복법은 행렬의 희박 구조를 보존한다.

## LU 분해

먼저 $n \times n$ 선형 연립방정식은 $Ax = b$ 형태를 가지며, 이 때 계수 행렬 A 는 다음의 형태를 가진다.

\begin{equation}
A = \left[ \begin{array}{}
a_{11} & a_{12} & a_{13} & \dots & a_{1n} \\
a_{21} & a_{22} & a_{23} & \dots & a_{1n} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & a_{n3} & \dots & a_{nn}
\end{array} \right]
\end{equation}

$A$에 순수 가우스 알고리즘을 적용함으로써 $A$를 단순한 두 행렬의 곱으로 분해할 수 있다.


\begin{equation}
L = \left[ \begin{array}{}
1 &  &  &  &  \\
l_{21} & 1 &  &  &  \\
l_{31} & l_{32} & 1 &  &  \\
\vdots & \vdots & \vdots & \ddots &  \\
l_{n1} & l_{n2} & l_{n3} & \dots & 1
\end{array} \right],
U = \left[ \begin{array}{}
u_{11} & u_{12} & u_{13} & \dots & u_{1n} \\
 & u_{22} & u_{23} & \dots & u_{2n} \\
 &  & u_{33} & \dots & u_{3n} \\
 & &  & \ddots & \vdots \\
 &  &  &  & u_{nn}
\end{array} \right]
\end{equation}

$LU$분해를 하기 위해 전진 소거 과정을 거치고 이 과정에서 여러 개의 $M$ 하삼각행렬을 얻게 된다.

$A$를 상삼각행렬로 만들기 위해 여러 개의 $M$을 곱하게 되면 다음의 식을 얻게 된다.

$$M_{3}M_{2}M_{1}Ax = M_{3}M_{2}M_{1}b$$

이제 전진 소거 과정이 완료되었으며 $M = M_{3}M_{2}M_{1}$을 통해 $LU$분해를 얻을 수 있다.

추가로 $LU$ 분해 구조는 알고리즘에서 0으로 나누는 경우가 없다는 점에 의존한다.
LU 분해를 수행하는 코드로 Doolittle factorization 이 있다.


In [60]:
#Doolittle factorization
def Doolittle(Arr):
  arrL = np.zeros_like(Arr)
  arrU = np.zeros_like(Arr)
  n = Arr.shape[0]
  for i in range(n):
    arrL[i, i] = 1
    for j in range(i, n):
      if i > 0:
        arrU[i, j] = Arr[i, j] - np.dot(arrL[i, :i], arrU[:i, j])
      if i==0:
        arrU[i, j] = Arr[i, j]

    for k in range(i+1, n):
      if i > 0:
        arrL[k, i] = (Arr[k, i] - np.dot(arrL[k, :i], arrU[:i, i])) / arrU[i, i]

      if i==0:
        arrL[k, i] = Arr[k, i]/arrU[i,i]
    #print('i = ', i, '\n', arrL, '\n', arrU)
  return arrL, arrU

tempa = np.array([[6, -2, 2, 4], [12, -8, 6, 10], [3, -13, 9, 3], [-6, 4, 1, -18]]).astype(np.float32)
[tempaL, tempaU] = Doolittle(tempa)
print(tempaL)
print(tempaU)
print(np.dot(tempaL, tempaU) == tempa)

[[ 1.   0.   0.   0. ]
 [ 2.   1.   0.   0. ]
 [ 0.5  3.   1.   0. ]
 [-1.  -0.5  2.   1. ]]
[[ 6. -2.  2.  4.]
 [ 0. -4.  2.  2.]
 [ 0.  0.  2. -5.]
 [ 0.  0.  0. -3.]]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]
